### Using pytorch kinematics
1. This is based on the repository from https://github.com/UM-ARM-Lab/pytorch_kinematics
2. Simply install and run `pip-compile requirements.toml` and `pip install -r requirements.txt`

In [49]:
import math
import torch
import pytorch_kinematics as pk

import os
from timeit import default_timer as timer

import time

d = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

chain = pk.build_serial_chain_from_urdf(open("./urdf/kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
chain = chain.to(dtype=dtype, device=d)

N = 400
th_batch = torch.rand(N, len(chain.get_joint_parameter_names()), dtype=dtype, device=d)
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)

########################### FK ###########################

t1 = timer()
# order of magnitudes faster when doing FK in parallel
# elapsed 0.008678913116455078s for N=1000 when parallel
# (N,4,4) transform matrix; only the one for the end effector is returned since end_only=True by default
tg_batch = chain.forward_kinematics(th_batch)

t2 = timer()

# elapsed 8.44686508178711s for N=1000 when serial
for i in range(N):
    tg = chain.forward_kinematics(th_batch[i])
t3 = timer()

print("Execution time for FK, [parallel] = {}, [sequential] = {}".format(t2-t1, t3-t2))

########################### JACOBIAN ###########################

t1 = timer()
J = chain.jacobian(th)
t2 = timer()

print("Execution time for Jac, [parallel] = {}".format(t2-t1))

########################### INVERSE KINEMATICS ###########################
# transformation from robot base frame to world frame
pos = torch.tensor([0.0, 0.0, 0.0], device=d)
rot = torch.tensor([0.0, 0.0, 0.0], device=d)
rob_tf = pk.Transform3d(pos=pos, rot=rot, dtype=dtype, device=d)

# world frame goals
# generate random goal joint angles (so these are all achievable)
# use the joint limits to generate random joint angles
lim = torch.tensor(chain.get_joint_limits(), device=d)
goal_q = torch.rand(N, 7, dtype=dtype, device=d) * (lim[1] - lim[0]) + lim[0]

# get ee pose (in robot frame)
goal_in_rob_frame_tf = chain.forward_kinematics(goal_q)

# transform to world frame for visualization
goal_tf = rob_tf.compose(goal_in_rob_frame_tf)
goal = goal_tf.get_matrix()
goal_pos = goal[..., :3, 3]
goal_rot = pk.matrix_to_euler_angles(goal[..., :3, :3], "XYZ")

ik = pk.PseudoInverseIK(chain, max_iterations=30, num_retries=10,
                        joint_limits=lim.T,
                        early_stopping_any_converged=True,
                        early_stopping_no_improvement="all",
                        # line_search=pk.BacktrackingLineSearch(max_lr=0.2),
                        debug=False,
                        lr=0.2)

t1 = timer()
sol = ik.solve(goal_in_rob_frame_tf)
t2 = timer()
print("Execution time for IK, [parallel] = {}".format(t2-t1))
print("IK converged number: %d / %d" % (sol.converged.sum(), sol.converged.numel()))
print("IK took %d iterations" % sol.iterations)
print("IK solved %d / %d goals" % (sol.converged_any.sum(), N))

Execution time for FK, [parallel] = 0.0017108179999922868, [sequential] = 0.4086778900000354
Execution time for Jac, [parallel] = 0.005182711000088602
Execution time for IK, [parallel] = 0.2590081399994233
IK converged number: 882 / 4000
IK took 30 iterations
IK solved 371 / 400 goals
